Command Profile in TVB
----------------------

This is a simple demo showing for to interract with TVB in the command profile.

More examples can be found here:
https://github.com/the-virtual-brain/tvb-framework/tree/master/tvb/interfaces/command/demos

In [1]:
from tvb.basic.profile import TvbProfile
TvbProfile.set_profile(TvbProfile.COMMAND_PROFILE)
from tvb.interfaces.command.lab import *

2016-04-23 16:08:09,319 - INFO - tvb.interfaces.web.mplh5.mplh5_server - MPLH5 back-end server started.


In [2]:
list_projects()

                    name    id
         Default_Project     1


Create a new project to test with

In [3]:
proj = new_project("sandbox")

In [4]:
proj

<Project('sandbox', '2')>

In [5]:
list_datatypes(proj.id)

                    type              tag    id                              gid         date


Nothing! Let's import a connectivity

In [7]:
import os
import tvb_data
p = os.path.join(os.path.dirname(tvb_data.__file__), 'connectivity/connectivity_66.zip')
import_conn_zip(proj.id, p)

2016-04-23 16:09:40,769 - INFO - tvb.core.services.flow_service - Starting operation ZIPConnectivityImporter
2016-04-23 16:09:40,974 - INFO - tvb.core.services.flow_service - Finished operation:ZIPConnectivityImporter


In [8]:
list_datatypes(proj.id)

                    type              tag    id                              gid         date
            Connectivity             None    33 a39c12e1-0954-11e6-94a8-3c075431bf56 2016-04-23 16:09:40.877940


Now let's grab that connectivity from the database. 
IMPORTANT: Make sure to adjunt the DT id bellow:

In [9]:
conn, = load_dt(33)
conn

<DataType(33, a39c12e1-0954-11e6-94a8-3c075431bf56, Connectivity, tvb.datatypes.connectivity, John Doe, RAW_DATA, None, None, None, None, None, None)>

and run a simulation with it

In [16]:
params = dict(connectivity=conn)
sim_op = fire_simulation(project_id=proj.id, connectivity=conn.gid, simulation_length=100)

2016-04-23 16:16:47,680 - INFO - tvb.core.services.flow_service - Starting operation SimulatorAdapter
2016-04-23 16:16:47,706 - INFO - tvb.core.services.flow_service - Finished operation:SimulatorAdapter


We should wait for the simulation to finish, and afterwards, TimeSeries should also be part of this project.

Refresh the Web GUI to see the status of your operation.

In [19]:
list_datatypes(proj.id)

                    type              tag    id                              gid         date
            Connectivity             None    33 a39c12e1-0954-11e6-94a8-3c075431bf56 2016-04-23 16:09:40.877940
         SimulationState             None    34 8b523330-0955-11e6-9251-3c075431bf56 2016-04-23 16:16:09.600679
         SimulationState             None    35 a51b2b6e-0955-11e6-a02d-3c075431bf56 2016-04-23 16:16:52.858382
        TimeSeriesRegion             None    36 8b50c4dc-0955-11e6-8237-3c075431bf56 2016-04-23 16:16:55.278417
        TimeSeriesRegion             None    37 a519bcfa-0955-11e6-90d0-3c075431bf56 2016-04-23 16:16:57.903965
